In [1]:
import pandas as pd
import plotly.graph_objects as go
import sqlite3

# Connect to the database
conn = sqlite3.connect('../../../data/stocks_analysis.db')

# Query the combined filtered data from the database
combined_filtered_data = pd.read_sql_query("SELECT * FROM combined_filtered_data", conn)

# Disconnect from the database
conn.close()

# Define the list of oil companies
oil_companies = ['ExxonMobil', 'Chevron', 'ConocoPhillips']  # Update with the actual names used in your dataset

# Filter the data to include only oil companies
oil_data = combined_filtered_data[combined_filtered_data['Company'].isin(oil_companies)]

# Available columns (metrics) to plot
available_columns = ['Adj Close', 'Volume', 'Open', 'High', 'Low']

# Set default metric (can be changed via dropdown later)
metric = 'Adj Close'

In [2]:
# Create an empty figure object
fig = go.Figure()

# Add traces for each oil company using the default metric (initially set to 'Adj Close')
for company in oil_data['Company'].unique():
    company_data = oil_data[oil_data['Company'] == company]
    fig.add_trace(go.Scatter(x=company_data['Date'], y=company_data[metric], mode='lines', name=company))

# Create dropdown menu for selecting different metrics
dropdown_buttons = [
    dict(
        label=col,
        method="update",
        args=[{"y": [oil_data[oil_data['Company'] == company][col] for company in oil_data['Company'].unique()]}]
    ) for col in available_columns
]

# Create layout with dropdown
fig.update_layout(
    xaxis_title="Date",
    legend=dict(
        orientation="v",
        y=0.5,
        x=1.1
    ),
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
            x=0.17,  # Adjust the position of the dropdown
            xanchor="left",
            y=1.2,
            yanchor="top"
        )
    ]
)

# Display the plot
fig.show()

In [3]:
# Save the oil plot as an HTML file
fig.write_html('oil_plotly.html')
